In [65]:
from classes.Utils import *
from classes.CRV import *
from classes.Corpus import *
from classes.Vectorizer import *

In [66]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
corpus_removal_threshold     = 1
vectorizer_removal_threshold = 2
grouping_similarity          = .6
filepath = 'recipes'

In [ ]:
corpus = Corpus(
    filepath          = f'corpora/{filepath}',
    spell_correct     = 'corpora/spelling_dictionary',
    lemmatize         = False,
    removal_threshold = corpus_removal_threshold)

vectorizer = corpus.create_vectorizer(removal_threshold = vectorizer_removal_threshold)

Data collected:
    - 12981 unique words found.
    - most common words: ['.', 'the', 'i', 'and', '<END>']
    - least common words: ['yummy', 'zen', 'ziploc', 'µ', 'ë']


Corpus loaded:
    - 63969 sentences.
    - longest sentence: 

<START> heads - up : this is a long review and includes suggestions for improvements that i personally would make as well as improvements or modifications that i feel are necessary to avoid having an annoying costume . overall this is a pretty great costume and really only needs a couple of modifications ( tack sew the collar so it isn ' t floppy , make padded arm bands so the sleeves actually poof out , and use a different petticoat ) . read on for a lot more detail . ( and check back for an updated review including before and after pics once i have made the modifications i feel are necessary ) < br / > < br / > as an added note for my kid friendly disney <UNK> : the provided hoop skirt is very much not kid friendly . if you are the type to sit on the f

In [ ]:
# get matrix of similarities between all words

twentieth = vectorizer.vsize // 20
differences = np.zeros((vectorizer.vsize, vectorizer.vsize,))

for i in range(vectorizer.vsize):
    word_diffs = keras.ops.einsum('v,bv->bv', vectorizer.matrix[i], vectorizer.matrix)
    word_diffs = keras.ops.sqrt(word_diffs)
    word_diffs = keras.ops.sum(word_diffs, axis = -1)
    word_diffs *= word_diffs
    differences[i] = word_diffs.numpy()
    
    if (i % twentieth == 0):
        print('twentieth')

differences *= (1 - np.eye(differences.shape[0], differences.shape[1]))
similar_words = np.asarray(differences >= grouping_similarity)


twentieth


KeyboardInterrupt: 

In [ ]:
coords = np.array(similar_words.nonzero())
print(len(coords[0]))

21160


In [ ]:
# https://www.geeksforgeeks.org/dsa/maximal-clique-problem-recursive-solution/

def bron_kerbosch(R, P, X, graph):
    if not P and not X:
        yield R
    while P:
        v = P.pop()
        yield from bron_kerbosch(
            R.union({v}),
            P.intersection(graph[v]),
            X.intersection(graph[v]),
            graph
        )
        X.add(v)


edges = [(int(coords[0,i])+1, int(coords[1,i])+1) for i in range(coords.shape[1])]
n = len(corpus.vocab)  # Number of nodes

# Create an adjacency list from the edges
graph = {i: set() for i in range(1, n + 1)}
for u, v in edges:
    graph[u].add(v)
    graph[v].add(u)
# Graph [x] is all words that are near x

# Convert set keys into sorted lists for consistent ordering
graph = {key: set(graph[key]) for key in graph}
# Graph [x] is now a set still, not a list. ???

# Calling bron_kerbosch with an empty set, the graph keys (range(1, n+1)), and the whole graph
all_cliques = list(bron_kerbosch(set(), set(graph.keys()), set(), graph))

# sort and clip all len-1 cliques
all_cliques = list(sorted([clique for clique in all_cliques if len(clique) > 1], key = lambda x: -len(x)))

print(f'Cliques Found: {len(all_cliques)}; Longest Clique: {len(max(all_cliques, key = len))}')


KeyboardInterrupt: 

In [ ]:
# for clique in all_cliques:
#     print(set(vectorizer.to_str(idx-1) for idx in clique))

In [ ]:
# write to a file

with open(f'discoveries/cliques/{filepath}.txt', 'w') as f:

    cliques_string = '\n'.join([str(set(vectorizer.to_str(idx-1) for idx in clique)) for clique in all_cliques])

    cliques_string = f'''Corpus Removal Threshold: {corpus_removal_threshold}\n
Vectorizer Removal Threshold: {vectorizer_removal_threshold}\n
Grouping Similarity: {grouping_similarity}\n
                         \n''' + cliques_string
    
    f.write(cliques_string)

In [ ]:
search_word = '0'

if search_word not in corpus.vocab:
    raise Exception("Word not in vocabulary")

word_index = vectorizer.to_int(search_word) + 1
found = False

for clique in all_cliques:
    if word_index in clique:
        found = True
        print(set(vectorizer.to_str(idx-1) for idx in clique))

if not found:
    print('None found')

Exception: Word not in vocabulary